In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import matplotlib.image as mpimg
import os
import sys
import cv2
from tqdm import tqdm
from functools import partial
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torchvision.transforms import v2

sys.path.append(os.path.join(os.getcwd(), "uzk_cvproject"))
from fly_dataset import FLYDataset
from fly_cnn import CNN_Fly, ResNet50Keypoints 

In [73]:
# Visualize
def fly_predict(model, dataset, idx, device="cuda"):
    model.eval()
    img, gt = dataset[idx]
    img = img.to(device)

    with torch.no_grad():
        pred_kp, _, _, _ = model(img)
    print(pred_kp.shape)

    return img



In [74]:
def fly_visualize(img, annotations, gt = None):
    fig, axes = plt.subplots(1,2)
    ax = axes[0]
    ax.imshow(img, cmap="gray")
    ax.axis("off")
    ax.scatter(annotations[0], annotations[1], c="r", label="Pred", s=10)
    if gt != None:
        ax.scatter(gt[0], gt[1], c="g", label="GT", s=10, alpha=0.6)
    plt.show()


In [ ]:
model = CNN_Fly((480, 960), 32)
train_dataset = FLYDataset("/scratch/cv-course2025/group2/data")
test_dataset = FLYDataset("/scratch/cv-course2025/group2/data", mode="test")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Use CPU for debug so you do not use too much GPU time
print(f"Used Device: {device}")

model.load_state_dict(torch.load("./fly-test-resnet50.pt"))
model.to(device)
img, anno = test_dataset.__getvisual__(0)

img_model = fly_predict(model, test_dataset, 0)

fly_visualize(img, anno)

Used Device: cuda:0


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 3, 480, 960] to have 1 channels, but got 3 channels instead